In [1]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from deepmatch.models import YoutubeDNN
from deepmatch.utils import sampledsoftmaxloss
warnings.filterwarnings('ignore')

DeepCTR version 0.8.7 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.8.7


## 读取数据

In [2]:
data_path = './data/SMP新数据-20210607-20210702/'
save_path = './temp_results/'
# 做召回评估的一个标志, 如果不进行评估就是直接使用全量数据进行召回
metric_recall = True

In [3]:
# debug模式： 从训练集中划出一部分数据来调试代码
def get_sample(data_path, sample_nums=10000):
    """
        训练集中采样一部分数据调试
        data_path: 原数据的存储路径
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = pd.read_csv(data_path + 'orders_train.txt', sep='\t')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    # all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果应该讲测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_df(data_path, offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'orders_train.txt', sep='\t')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)
    
    # all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

In [4]:
all_train_df = get_all_df(data_path, offline=True)

In [5]:
all_train_df.head()

,user_id,wm_order_id,wm_poi_id,aor_id,order_price_interval,order_timestamp,ord_period_name,order_scene_name,aoi_id,takedlvr_aoi_type_name,dt
0,178557,0,2334,6,<29,1623061539,3,0,NaN,未知,20210607
1,175118,1,3315,0,<29,1623032193,1,1,0.0,0,20210607
2,36208,2,2168,0,<29,1623036350,1,0,1.0,0,20210607
3,102798,3,3071,0,"[29,36)",1623071723,4,0,2.0,0,20210607
4,73712,4,2902,0,"[49,65)",1623020472,0,2,3.0,1,20210607


In [ ]:
# 读取相似度矩阵

i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

## 工具函数

In [5]:
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

In [6]:
# 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
def get_user_poi_time(click_df):
    
    click_df = click_df.sort_values('order_timestamp')
    
    def make_item_time_pair(df):
        return list(zip(df['wm_poi_id'], df['order_timestamp']))
    
    user_item_time_df = click_df.groupby('user_id')['wm_poi_id', 'order_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

In [7]:
# 根据时间获取商品被点击的用户序列  {item1: [(user1, time1), (user2, time2)...]...}
# 这里的时间是用户点击当前商品的时间，好像没有直接的关系。
def get_item_user_time_dict(click_df):
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['order_timestamp']))
    
    click_df = click_df.sort_values('order_timestamp')
    item_user_time_df = click_df.groupby('wm_poi_id')['user_id', 'order_timestamp'].apply(lambda x: make_user_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'user_time_list'})
    
    item_user_time_dict = dict(zip(item_user_time_df['wm_poi_id'], item_user_time_df['user_time_list']))
    return item_user_time_dict

In [8]:
# 获取当前数据的历史点击和最后一次点击 (划分验证集)
def get_hist_and_last_click(all_click):
    
    all_click = all_click.sort_values(by=['user_id', 'order_timestamp'])
    click_last_df = all_click.groupby('user_id').tail(1)

    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

In [9]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['wm_order_id'].value_counts().index[:k]
    return topk_click

In [10]:
########################### 评估函数 ##############################
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=50):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['wm_order_id']))
    user_num = len(user_recall_items_dict)
    
    for k in range(10, topk+1, 10):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            # 获取前k个召回的结果
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

In [28]:
############################## 生成提交结果 ###############################
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['wm_order_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['wm_order_id'])['pred_score'].rank(ascending=False, method='first')

    # 判断是不是每个用户都有5篇文章及以上
    # tmp = recall_df.groupby('wm_order_id').apply(lambda x: x['rank'].max())
    # assert tmp.min() >= topk

    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['wm_order_id', 'rank']).unstack(-1).reset_index()

    # submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    # submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
    #                                               3: 'article_3', 4: 'article_4', 5: 'article_5'})

    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.txt'
    submit.to_csv(save_name, sep='\t', index=False, header=False)

## 计算相似度

### itemCF i2i

基于用户的交互序列，CF 得到 i2i 相似度

In [57]:
# all_train_df = get_train_sample(data_path)
all_train_df_tsnorm = all_train_df.copy()
all_train_df_tsnorm['order_timestamp'] = all_train_df_tsnorm[['order_timestamp']].apply(max_min_scaler)
get_user_poi_time(all_train_df_tsnorm)

{0: [(3263, 0.022690520676598488),
  (2972, 0.07172517182272635),
  (809, 0.11823057039082437),
  (2972, 0.2619316897876398),
  (806, 0.30640497803645345),
  (1738, 0.35699995039600524),
  (2183, 0.4033769297331856),
  (474, 0.4558860651355567),
  (2972, 0.5019334534852318),
  (956, 0.5091133561511709),
  (786, 0.592460192794193),
  (2087, 0.64048953631288),
  (956, 0.6495196679839284),
  (2438, 0.690229115340311),
  (1975, 0.784837712263761),
  (2972, 0.8812513434415251),
  (1842, 0.9292933635366546),
  (2971, 0.9760539471000953),
  (2971, 0.9760649702100453)],
 1: [(3374, 0.11764579440797632),
  (2624, 0.4158997337918947),
  (2624, 0.5104835287179572),
  (2624, 0.6072107673737992),
  (2398, 0.6547159620143631),
  (2725, 0.7559472433957792),
  (2986, 0.9408544012522253)],
 2: [(570, 0.2231534912944989),
  (2417, 0.4029244310697377),
  (1848, 0.6513351741926949),
  (2856, 0.6515269763058251),
  (3116, 0.8864790533353175),
  (3231, 0.8872142947689832),
  (618, 0.9836797345635124)],
 3: 

In [58]:
def itemcf_sim(df): # , item_created_time_dict
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
    """
    
    user_item_time_dict = get_user_poi_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素
        for loc1, (i, i_click_time) in enumerate(item_time_list):
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for loc2, (j, j_click_time) in enumerate(item_time_list):
                if(i == j):
                    continue
                    
                # 考虑文章的正向顺序点击和反向顺序点击    
                loc_alpha = 1.0 if loc2 > loc1 else 0.7
                # 位置信息权重，其中的参数可以调节
                loc_weight = loc_alpha * (0.9 ** (np.abs(loc2 - loc1) - 1))
                # 点击时间权重，其中的参数可以调节
                click_time_weight = np.exp(0.7 ** np.abs(i_click_time - j_click_time))
                # # 两篇文章创建时间的权重，其中的参数可以调节
                # created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                i2i_sim[i].setdefault(j, 0)
                # 考虑多种因素的权重计算最终的文章之间的相似度
                i2i_sim[i][j] += loc_weight * click_time_weight / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_

In [59]:
i2i_sim = itemcf_sim(all_train_df_tsnorm)

100%|██████████| 200000/200000 [00:59<00:00, 3363.20it/s]


In [60]:
# list(i2i_sim.items())[0]
from scipy.sparse import coo_matrix
rows, cols, data = [], [], []

nuser, npoi = all_train_df['user_id'].max()+1, all_train_df['wm_poi_id'].max()+1
for i, row in i2i_sim.items():
    for j, score in row.items():
        data += [score]
        cols.append(j)
        rows.append(i)
coo = coo_matrix((data, (rows, cols)), shape=(npoi, npoi))

In [61]:
max(cols), npoi, max(rows)

(3576, 3577, 3576)

In [62]:
scores = coo.toarray()
scores

array([[0.        , 0.00373839, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00534056, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [63]:
scores.sum(axis=1)

array([4.48124859, 3.22746572, 2.31173045, ..., 0.35679115, 2.08088054,
       0.66076759])

In [64]:
all_train_df.loc[all_train_df['user_id']==178557].head()
all_train_df

,user_id,wm_order_id,wm_poi_id,aor_id,order_price_interval,order_timestamp,ord_period_name,order_scene_name,aoi_id,takedlvr_aoi_type_name,dt
0,178557,0,2334,6,<29,1623061539,3,0,NaN,未知,20210607
1,175118,1,3315,0,<29,1623032193,1,1,0.0,0,20210607
2,36208,2,2168,0,<29,1623036350,1,0,1.0,0,20210607
3,102798,3,3071,0,"[29,36)",1623071723,4,0,2.0,0,20210607
4,73712,4,2902,0,"[49,65)",1623020472,0,2,3.0,1,20210607
...,...,...,...,...,...,...,...,...,...,...,...
1071868,176924,1071868,3548,7,<29,1624791638,3,1,440.0,7,20210627
1071869,80609,1071869,1613,7,"[49,65)",1624788052,3,3,3032.0,9,20210627
1071870,31420,1071870,261,7,>=65,1624784877,3,1,NaN,未知,20210627
1071871,28455,1071871,3016,7,<29,1624776705,2,6,428.0,7,20210627


In [81]:
# 注意这里计算的 i2i 相似度是不对称的
# 1. 用户连续购买之间的相似度
i = 2334
j = 3315
print(scores[i][j], scores[j][i])

# 2. 观察 session 相似度
# sess = '372#1327#1957#1988#2204#100#1327#2372#1817#1437#2760#2956#2034#2760#1327'
sess = '2267#1015#23999#29305#15534#18866#18866'
sess = sess.split('#')
for i in range(len(sess)):
    for j in range(i+1, len(sess)):
        print('%.4f' % scores[i][j], end=' ')
    print()

0.0015707058768432416 0.0022438655383474884
0.0037 0.0000 0.0102 0.0000 0.0038 0.0000 
0.0000 0.0079 0.0000 0.0000 0.0000 
0.0000 0.0011 0.0059 0.0000 
0.0023 0.0030 0.0000 
0.0053 0.0211 
0.0011 



In [73]:
scores.mean(), scores.max()

(0.0007849699154756643, 2.474044057704529)

可以看到，基于 CF 计算的 i2i 相似度，一定程度上也可以解释 session 序列；但也有 session 中相邻但是这里计算的相似度=0 的情况

### userCF u2u

基于用户的交互序列，CF 得到 u2u 相似度

In [8]:
# 用户活跃度作为关联规则
def get_user_activate_degree_dict(all_click_df):
    all_click_df_ = all_click_df.groupby('user_id')['wm_order_id'].count().reset_index()
    
    # 用户活跃度归一化
    mm = MinMaxScaler()
    all_click_df_['wm_order_id'] = mm.fit_transform(all_click_df_[['wm_order_id']])
    user_activate_degree_dict = dict(zip(all_click_df_['user_id'], all_click_df_['wm_order_id']))
    
    return user_activate_degree_dict

In [9]:
def usercf_sim(all_click_df, user_activate_degree_dict):
    """
        用户相似性矩阵计算
        :param all_click_df: 数据表
        :param user_activate_degree_dict: 用户活跃度的字典
        return 用户相似性矩阵
        
        思路: 基于用户的协同过滤(详细请参考上一期推荐系统基础的组队学习) + 关联规则
    """
    item_user_time_dict = get_item_user_time_dict(all_click_df)
    
    u2u_sim = {}
    user_cnt = defaultdict(int)
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        for u, click_time in user_time_list:
            user_cnt[u] += 1
            u2u_sim.setdefault(u, {})
            for v, click_time in user_time_list:
                u2u_sim[u].setdefault(v, 0)
                if u == v:
                    continue
                # 用户平均活跃度作为活跃度的权重，这里的式子也可以改善
                activate_weight = 100 * 0.5 * (user_activate_degree_dict[u] + user_activate_degree_dict[v])   
                u2u_sim[u][v] += activate_weight / math.log(len(user_time_list) + 1)
    
    u2u_sim_ = u2u_sim.copy()
    for u, related_users in u2u_sim.items():
        for v, wij in related_users.items():
            u2u_sim_[u][v] = wij / math.sqrt(user_cnt[u] * user_cnt[v])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(u2u_sim_, open(save_path + 'usercf_u2u_sim.pkl', 'wb'))

    return u2u_sim_

In [17]:
# 由于usercf计算时候太耗费内存了，这里就不直接运行了
# 如果是采样的话，是可以运行的
# all_train_df_tsnorm = all_train_df.copy()
# all_train_df_tsnorm['order_timestamp'] = all_train_df_tsnorm[['order_timestamp']].apply(max_min_scaler)

# user_activate_degree_dict = get_user_activate_degree_dict(all_train_df_tsnorm)
# u2u_sim = usercf_sim(all_train_df_tsnorm, user_activate_degree_dict)


NameError: name 'u2u_sim' is not defined

* 训练集中的 user数量为20000，而 item数量为 3577

In [7]:
all_train_df.user_id.max()

199999

## 召回

In [ ]:
# 由于这里需要做召回评估，所以讲训练集中的最后一次点击都提取了出来
# if not metric_recall:
#     user_multi_recall_dict['youtubednn_recall'] = youtubednn_u2i_dict(all_click_df, topk=20)
# else:
#     trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
#     user_multi_recall_dict['youtubednn_recall'] = youtubednn_u2i_dict(trn_hist_click_df, topk=20)
#     # 召回效果评估
#     metrics_recall(user_multi_recall_dict['youtubednn_recall'], trn_last_click_df, topk=20)

### itemcf recall

* 采用 item CF 进行召回，基于根据用户点击/购买序列计算得到的 i2i sim，
  * 在原始版本中用到了 **loc权重、文章内容相似度、文章创建时间差** 这些关联规则；
  * 迁移到这里，仅仅用到了 loc权重，后期考虑增加规则/调整超参

TODO 使用 YoutubeDNN 得到的 embed


In [13]:
# 基于商品的召回i2i
def item_based_recommend(
    user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, 
    # item_created_time_dict, emb_i2i_sim
    ):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: [(item1, time1), (item2, time2)..]...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全
        :param emb_i2i_sim: 字典基于内容embedding算的文章相似矩阵
        
        return: 召回的文章列表 [(item1, score1), (item2, score2)...]
    """
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
            
            # 文章创建时间差权重
            # created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # 相似文章和历史点击文章序列中历史文章所在的位置权重
            loc_weight = (0.9 ** (len(user_hist_items) - loc))
            
            # content_weight = 1.0
            # if emb_i2i_sim.get(i, {}).get(j, None) is not None:
            #     content_weight += emb_i2i_sim[i][j]
            # if emb_i2i_sim.get(j, {}).get(i, None) is not None:
            #     content_weight += emb_i2i_sim[j][i]
                
            item_rank.setdefault(j, 0)
            item_rank[j] += loc_weight * wij
                # created_time_weight * content_weight * 
                
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank

In [14]:
# 先进行itemcf召回, 为了召回评估，所以提取最后一次点击

if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_train_df)
else:
    trn_hist_click_df = all_train_df

user_recall_items_dict = collections.defaultdict(dict)
user_item_time_dict = get_user_poi_time(trn_hist_click_df)

i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))
# emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))

sim_item_topk = 20
recall_item_num = 20        # 基于 itemcf 召回的数量
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, 
                                                        # item_created_time_dict, emb_i2i_sim
                                                        )

# user_multi_recall_dict['itemcf_sim_itemcf_recall'] = user_recall_items_dict
pickle.dump(user_recall_items_dict, open(save_path + 'itemcf_recall_dict.pkl', 'wb'))

if metric_recall:
    # 召回效果评估
    metrics_recall(user_recall_items_dict, trn_last_click_df, topk=recall_item_num)

100%|██████████| 200000/200000 [04:51<00:00, 686.10it/s]


 topk:  10  :  hit_num:  0 hit_rate:  0.0 user_num :  200000
 topk:  20  :  hit_num:  0 hit_rate:  0.0 user_num :  200000


In [15]:
# 在测试集上运行，提交结果
d_test = pd.read_csv(data_path + "orders_test_poi.txt", encoding="utf-8", sep='\t')
test_recall_items_dict = collections.defaultdict(dict)
for user in tqdm(d_test['user_id'].unique()):
    test_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                    i2i_sim, sim_item_topk, recall_item_num, \
                                                    item_topk_click, 
                                                    # item_created_time_dict, emb_i2i_sim
                                                    )

100%|██████████| 91221/91221 [03:03<00:00, 496.42it/s]


In [22]:
results = []
for uid, poi in d_test[['user_id', 'wm_order_id']].values:
    for i, s in test_recall_items_dict[uid][:5]:
        results.append([poi, i, s])
results_df = pd.DataFrame(results, columns=['wm_order_id', 'wm_poi_id', 'pred_score'])
# 预测结果重新排序, 生成提交结果

In [27]:
results_df['wm_order_id'] = results_df['wm_order_id'].astype(int)
submit(results_df, topk=5, model_name='itemcf')